In [2]:
import torch
from torch.utils.data import TensorDataset, DataLoader  # these are needed for the training data
import numpy as np
from constants import *
# import optim
import torch.optim as optim
import torch.nn as nn # torch.nn allows us to create a neural network.
from torch.optim import Adam # optim contains many optimizers. This time we're using Adam


In [8]:
class Decoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.embedding_size = embedding_size
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.loss = nn.CrossEntropyLoss()
        print("from decoder init")
        print("adham")

    def forward(self, x):
        batch_size = x.shape[0]
        h0 = torch.zeros(1, batch_size, self.hidden_size).cuda()
        c0 = torch.zeros(1, batch_size, self.hidden_size).cuda()
        # print("from decoder forward")
        # print(x.shape)
        embeddings = self.embedding(x).cuda()
        # print("from decoder forward after embedding")
        # print(embeddings.shape)
        h, (hn, cn) = self.rnn(embeddings, (h0, c0))
        # h is the output of the RNN
        # hn is the hidden state of the last timestep
        # cn is the cell state of the last timestep
        out = self.fc(h)
        return out

In [4]:
from letters_dataset import LettersDataset
from train_collections import DS_ARABIC_LETTERS, DS_HARAKAT


dim_vocab = len(DS_ARABIC_LETTERS)
dim_out = len(DS_HARAKAT) + 2
embedding_dim = 64
n_epochs = 10
batch_size = 64
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataset = LettersDataset(device=device)
loader = DataLoader(dataset, shuffle=True, batch_size=batch_size)

# load val data
# da = LettersDataset('clean_out/X_val.csv', 'clean_out/y_val.csv')

w = 495


In [10]:
# create a decoder
model = Decoder(
    input_size= dim_vocab,
    hidden_size=256,
    output_size=dim_out,
    embedding_size=embedding_dim,
)
if torch.cuda.is_available():
   print("cuda is available")
   model = model.cuda()
#    x_train = x_train.cuda() 
#    y_train = y_train.cuda()

from decoder init
adham
cuda is available


In [11]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()
num_batches = len(loader)
print("Number of batches:", num_batches)
best_model = None
best_loss = np.inf
for epoch in range(n_epochs):
    model.train()
    for i, (X_decoder, Y_batch) in enumerate(loader):
        y_pred = ''
        y_pred = model(X_decoder)
        y_pred = y_pred.transpose(1, 2)
        loss = loss_fn(y_pred, Y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i % 100 == 0:
            print("Epoch %d, batch %d: Loss = %.4f" % (epoch, i, loss))

Number of batches: 2590
Epoch 0, batch 0: Loss = 2.6960
Epoch 0, batch 100: Loss = 0.1495
Epoch 0, batch 200: Loss = 0.1011
Epoch 0, batch 300: Loss = 0.0915
Epoch 0, batch 400: Loss = 0.1044
Epoch 0, batch 500: Loss = 0.0824
Epoch 0, batch 600: Loss = 0.0879
Epoch 0, batch 700: Loss = 0.0862
Epoch 0, batch 800: Loss = 0.0788
Epoch 0, batch 900: Loss = 0.0584
Epoch 0, batch 1000: Loss = 0.0624
Epoch 0, batch 1100: Loss = 0.0635
Epoch 0, batch 1200: Loss = 0.0670
Epoch 0, batch 1300: Loss = 0.1363
Epoch 0, batch 1400: Loss = 0.0645
Epoch 0, batch 1500: Loss = 0.0552
Epoch 0, batch 1600: Loss = 0.0630
Epoch 0, batch 1700: Loss = 0.0694
Epoch 0, batch 1800: Loss = 0.0530
Epoch 0, batch 1900: Loss = 0.0804
Epoch 0, batch 2000: Loss = 0.0727
Epoch 0, batch 2100: Loss = 0.0921
Epoch 0, batch 2200: Loss = 0.0549
Epoch 0, batch 2300: Loss = 0.0598
Epoch 0, batch 2400: Loss = 0.0656
Epoch 0, batch 2500: Loss = 0.0666
Epoch 1, batch 0: Loss = 0.0589
Epoch 1, batch 100: Loss = 0.0436
Epoch 1, bat

In [12]:
# load X_test and y_test
X_TEST_PATH = f'clean_out/X_val.csv'
Y_TEST_PATH = f'clean_out/y_val.csv'
test_dataset_tensor = LettersDataset(input_data_file=X_TEST_PATH, output_data_file=Y_TEST_PATH, device=device)

# create a data loader
test_data_loader = DataLoader(test_dataset_tensor, batch_size=64, shuffle=True)

w = 500


In [14]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for (X_decoder, Y_batch) in test_data_loader:
        is_padding = (X_decoder == test_dataset_tensor.char_encoder.get_pad_token())
        y_pred = model(X_decoder)
        y_pred = y_pred.transpose(1, 2)
        _, predicted = torch.max(y_pred.data, 1)
        # Count only non-padding characters
        total += torch.sum(~is_padding).item()

        # Count correct predictions
        correct += torch.sum((predicted == Y_batch) & (~is_padding)).item()
print("Accuracy: %.2f%%" % (100 * correct / total))

Accuracy: 86.87%


In [ ]:
class Pipeline():
    def __init__(self, train_data_path, test_data_path, device):
        self.train_data_path = train_data_path
        self.test_data_path = test_data_path
        self.device = device
        self.dataset = LettersDataset(input_data_file=self.train_data_path, output_data_file=self.test_data_path, device=self.device)
        self.decoder = Decoder(
            input_size= dim_vocab,
            hidden_size=256,
            output_size=dim_out,
            embedding_size=embedding_dim,
        )

    def predict(self, X):
        y_pred = self.decoder(X)
        y_pred = y_pred.transpose(1, 2)
        _, predicted = torch.max(y_pred.data, 1)
        return predicted